# AI Makerspace Session 3
## Title: Running OLLAMA on HPC
_Hosted by:_ Mithun Paul
_Date:_ 10/18/24


steps
- Run Ollama on laptop
- how to connect to HPC
- how to install ollama on HPC

- Run Ollama on Laptop
# # What is Ollama

- (Omni-Layer Learning Language Acquisition Model)
- Refer earlier [lecture](https://github.com/ua-datalab/Generative-AI/wiki/Running-LLM-Locally:-Ollama) from other workshop.


### for osx -after downloading in OSX, move to applications folder, double click to start, then in a fresh terminal type
`ollama run llama3.2`


### Connecting to HPC
- [OOD](https://ood.hpc.arizona.edu/pun/sys/dashboard)
- SSH


# SSH commands to connect to HPC- 

- `ssh mithunpaul@hpc.arizona.edu`
        - this connects to Login node (to know what login node and bastion node is refer HPC [docs](https://hpcdocs.hpc.arizona.edu/quick_start/logging_in/#system-access)

now to connect to Bastion hosts (wentletrap and junonia)type
- `shell`
  
Then switch to elGato (easier access)

- `elgato`
- `tmux`

COmmands to install [Ollama](https://github.com/ollama/ollama/blob/main/docs/linux.md#manual-install)

- `curl -L https://ollama.com/download/ollama-linux-amd64.tgz -o ollama-linux-amd64.tgz`
- `tar -zxvf ollama-linux-amd64.tgz`
-  `./bin/ollama serve`

  Switch out from that tmux window, or connect through another sessionto HPC
  - from the other tmux window run
  -  `./bin/ollama run llama3.2`